In [1]:
from pymongo import MongoClient
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import string
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
client = MongoClient("mongodb://localhost:27017/")
db = client["Arxiv"]
collection = db["Arxiv Papers"]
summary = [[doc["summary"]] for doc in collection.find()]

In [11]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

In [12]:
lemmatizer = WordNetLemmatizer()

In [13]:
def preprocessing(summaries):
    summaries[0] = summaries[0].lower()
    summaries[0] = summaries[0].translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(summaries[0])
    pos_tags = pos_tag(tokens)
    lemmatized_tokens = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos))
        for word, pos in pos_tags
    ]
    return [" ".join(lemmatized_tokens)]

In [15]:
for i in range(len(summary)):
    summary[i] = preprocessing(summary[i])

In [16]:
train_size = int(0.9 * len(summary))
train_data = summary[:train_size]
test_data = summary[train_size:]

In [19]:
train_fixed = [train_data[index][0] for index in range(len(train_data))]

In [20]:
tfidf_vectorizer = TfidfVectorizer(lowercase=True, stop_words='english')

In [21]:
tfidf_mat = tfidf_vectorizer.fit_transform(train_fixed)

In [22]:
feature_names = tfidf_vectorizer.get_feature_names_out()
print(tfidf_mat.shape)

(7920, 26999)


In [24]:
new_tfidf = tfidf_vectorizer.transform([test_data[-1][0]])

In [25]:
similarities = cosine_similarity(new_tfidf, tfidf_mat)
most_similar = similarities.argmax()
score = similarities.max()

In [27]:
print(most_similar)
print(test_data[-1][0])
print(train_data[most_similar][0])
print(score)

6193
we present vbpimixtures an algorithm design to enhance the accuracy of phylogenetic posterior distribution particularly for treetopology and branchlength approximation despite the variational bayesian phylogenetic inference vbpi a leadingedge blackbox variational inference bbvi framework achieve remarkable approximation of these distribution the multimodality of the treetopology posterior present a formidable challenge to samplingbased learn technique such a bbvi advance deep learn methodology such a normalizing flow and graph neural network have be explore to refine the branchlength posterior approximation yet effort to ameliorate the posterior approximation over tree topology have be lack our novel vbpimixtures algorithm bridge this gap by harness the late breakthrough in mixture learning within the bbvi domain a a result vbpimixtures be capable of capture distribution over treetopologies that vbpi fail to model we deliver stateoftheart performance on difficult density estimatio